Documentation tensorflow:
* [Tensorflow Datasets](https://github.com/tensorflow/datasets)
* [Nouvelle doc tensorflow](https://www.tensorflow.org/guide/datasets)

* [Why tf.data is much better than feed_dict and how to build a simple data pipeline in 5 minutes.](https://dominikschmidt.xyz/tensorflow-data-pipeline/)

In [17]:
"""
enable_eager execution bloque tf.placeholder()

"""

import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [18]:
mnist_test

<MapDataset shapes: ((28, 28, 1), (10,)), types: (tf.uint8, tf.float32)>

In [19]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits
n_hidden_layer = 256 # layer number of features

In [20]:
# one hot encode for 10 MNIST classes
def my_one_hot(feature, label):
    return feature, tf.one_hot(label, depth=n_classes)

# load your data from tfds
mnist_data, train_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_data["train"], mnist_data["test"]

# convert your labels in one-hot
mnist_train = mnist_train.map(my_one_hot)
mnist_test = mnist_test.map(my_one_hot)

# you can batch your data here
mnist_train = mnist_train.batch(8)

# Ensure that the next batch is always immediately available to the GPU
# And reduces GPU starvation
mnist_train = mnist_train.prefetch(buffer_size=1)

In [21]:
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']),\
    biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)


In [28]:
"""
Modified version v1
"""
# Initializing the variables
init = tf.global_variables_initializer()

iterator = mnist_train.make_initializable_iterator()
batch_of_images = iterator.get_next()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        sess.run(train_iterator.initializer)
        try:
            # Go through the entire dataset
            while True:
                image_batch = sess.run(batch_of_images)
                
        except tf.errors.OutOfRangeError:
            print('End of Epoch.')

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext_6 (defined at <ipython-input-28-fc30c9598964>:8)  = IteratorGetNext[output_shapes=[[?,28,28,1], [?,10]], output_types=[DT_UINT8, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](IteratorV2_6)]]

Caused by op 'IteratorGetNext_6', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505

In [26]:
"""
Modified version v0
"""
# Initializing the variables
init = tf.global_variables_initializer()

iterator = mnist_train.make_initializable_iterator()
batch_of_images = iterator.get_next()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # make an iterator

    # Training cycle
    for epoch in range(training_epochs):

        sess.run(train_iterator.initializer)
        #batch_train_x, batch_train_y = sess.run(next_element)
        

        total_batch = int(train_info.splits["train"].num_examples/batch_size)
        image_batch = sess.run(batch_of_images)
        # Loop over all batches
        for i in range(total_batch):
            # Run optimization op (backprop) and cost op (to get loss value)
            #sess.run(optimizer, feed_dict={x: batch_train_x, y: batch_train_y})
            sess.run(optimizer, image_batch)

        # Calculate accuracy for test dataset
        #test_accuracy = sess.run(
        #    accuracy,
        #    mnist_test)

    #print('Test Accuracy: {}'.format(test_accuracy))

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext_4 (defined at <ipython-input-26-68b8495d72fe>:6)  = IteratorGetNext[output_shapes=[[?,28,28,1], [?,10]], output_types=[DT_UINT8, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](IteratorV2_3)]]

Caused by op 'IteratorGetNext_4', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # make an iterator
    train_iterator = mnist_train.make_initializable_iterator()
    next_element = train_iterator.get_next()

    # Training cycle
    for epoch in range(training_epochs):

        sess.run(train_iterator.initializer)
        batch_train_x, batch_train_y = sess.run(next_element)

        total_batch = int(train_info.splits["train"].num_examples/batch_size)

        # Loop over all batches
        for i in range(total_batch):
            # Run optimization op (backprop) and cost op (to get loss value)
            #sess.run(optimizer, feed_dict={x: batch_train_x, y: batch_train_y})
            sess.run(optimizer, feed_dict={x: batch_train_x, y: batch_train_y})

        # Calculate accuracy for test dataset
        test_accuracy = sess.run(
            accuracy,
            feed_dict={features: test_features, labels: test_labels})

    print('Test Accuracy: {}'.format(test_accuracy))

In [ ]:
mnist_example, = mnist_train.take(1)
image, label = mnist_example["image"], mnist_example["label"]

plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
print("Label: %d" % label.numpy())

In [ ]:
print(info.features)
print(info.features["label"].num_classes)
print(info.features["label"].names)

In [ ]:
print(info.splits["train"].num_examples)
print(info.splits["test"].num_examples)

In [ ]:
info = mnist_builder.info
print(info)

In [ ]:
train_ds = mnist_train.batch(128)

In [ ]:
train_ds

In [ ]:
import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits
n_hidden_layer = 256 # layer number of features

# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']),\
    biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(info.splits["train"].num_examples/batch_size)
        print(total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist_train.batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})